In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

def ask_chat_gpt(prompt):

    load_dotenv()
    api_key = os.getenv("OPENAI_KEY")

    client = OpenAI(
        api_key=api_key,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o",
    )
    message = chat_completion.choices[0].message.content
    return message

In [102]:
def make_algo_doc_gpt_promt(name):
    start_prompt = """
    Generate LaTeX code from the provided Python code. Follow these guidelines:

    1. **Document Structure**:
    - Start with `\\section{}` for the algorithm title.
    - Add a `\\subsection{Introduction}` for a brief overview of the algorithm.
    - Include a `\\subsection{Variables}` section that lists all variables with descriptions, using subscript notation (e.g., `v_{\\text{earth}}`).
    - Add a `\\subsection{Formulas}` section presenting the code's logic as LaTeX formulas. Use subscripted symbols for variable names instead of copying Python variable names directly.

    2. **Formatting Rules**:
    - Ensure that the output includes **only** the LaTeX content, without `\\documentclass`, `\\usepackage`, `\\begin{document}`, `\\end{document}`, or any unrelated text.
    - Do **not** include the triple backticks (e.g., ```latex or ```).
    - Properly close all LaTeX environments (e.g., `\\begin{align*}...\\end{align*}`).
    - Ensure all brackets, parentheses, and braces are matched correctly.
    - Maintain consistent subscript notation for all variables.

    3. **Important Notes**:
    - **Do not** include any text or explanations outside the LaTeX code.
    - Only the relevant LaTeX content for the `\\section`, `\\subsection`, `\\begin{align*}`, and `\\end{align*}` parts should be generated.
    - Ensure no extra or unrelated LaTeX sections are added.
    """

    file_path = f'python_code/{name}.py'

    # Open the file in read mode and save its contents to a string
    with open(file_path, 'r') as file:
        file_contents = file.read()

    total_prompt = start_prompt + file_contents
    print("Finishin making prompt")
    return total_prompt

def make_algo_example_gpt_promt(name):
    start_prompt = """
    Generate LaTeX code from the provided Python code. Follow these guidelines:

    1. **Document Structure**:
    - Use the provided LaTeX document and Python example to create an `Example` subsection.
    - Only include the variables in the same notation as used in the LaTeX document (e.g., `m_{\text{fuel}}` for `fuel_mass`).
    - Avoid any direct references to Python functions or libraries (e.g., do not use `np.linspace` or `np.argmin`).
    - You dont need to do the calculations, but can describe what the formulas do instead.

    2. **Formatting Rules**:
    - Include **only** the relevant LaTeX content, without `\\documentclass`, `\\usepackage`, `\\begin{document}`, `\\end{document}`, or any unrelated text.
    - Do **not** include triple backticks (e.g., ```latex or ```).
    - Use display math environments (e.g., `\\begin{align*}...\\end{align*}`) and ensure all brackets, parentheses, and braces are correctly matched.
    - Maintain consistent subscript notation for variables to match the example LaTeX document (e.g., `T_{\text{initial}}` for initial thrust).

    3. **Important Notes**:
    - Focus exclusively on formatting the variables in LaTeX with the same subscripts and symbols as in the provided document.
    - Do **not** include bullet points, itemized lists, or unrelated sections; limit content to the variables and the equations within the `align*` environment.
    - Provide ranges or lists in LaTeX notation rather than using Python functions (e.g., use `[5 \times 10^6, 2 \times 10^7]` instead of `np.linspace`).
    """

    latex_doc_path = f'algo_docs/{name}_doc.tex'
    with open(latex_doc_path, 'r') as file:
        latex_doc = file.read()

    algo_example_path = f'examples/{name}_example.py'
    print(algo_example_path)
    with open(algo_example_path, 'r') as file:
        algo_example = file.read()

    total_prompt = start_prompt + " Latex code: " + latex_doc  + " Python example" + algo_example
    print("Finishin making prompt")

    # Write the content to the file
    with open('total_prompt.txt', 'w') as file:
        file.write(total_prompt)
    return total_prompt

def make_algo_doc(name):
    total_prompt = make_algo_doc_gpt_promt(name)    
    print(f"Asking gpt to make doc for {name}")
    message = ask_chat_gpt(total_prompt)
    document_path_name = "algo_docs/" + name + "_doc.tex"
    with open(document_path_name, "w") as file:
        file.write(message)
    print(f"Finished making doc for {name}")

def make_algo_example(name):
    total_prompt = make_algo_example_gpt_promt(name)    
    print(f"Asking gpt to make example for {name}")
    message = ask_chat_gpt(total_prompt)
    document_path_name = "algo_docs/" + name + "_example.tex"
    with open(document_path_name, "w") as file:
        file.write(message)
    print(f"Finished making example for {name}")

In [ ]:
from pylatex import Document, Chapter, Subsection, NoEscape, Figure
from pylatex.utils import NoEscape
import subprocess
code_names = ["calculate_hohmann_transfer", "gravitational_assist"]
# Create and structure the LaTeX document programmatically
doc = Document(documentclass="report")

# Include custom preamble and metadata files
doc.preamble.append(NoEscape(r'\input{algo_docs/init.tex}'))  # Preamble with packages and styles
doc.append(NoEscape(r'\input{algo_docs/title_page.tex}'))     # Title page and table of contents
doc.append(NoEscape(r'\tableofcontents'))

# Create content sections
with doc.create(Chapter('Introduction')):
    doc.append('This document provides an overview of various algorithms and their significance in computational problem-solving. It aims to set the foundation for understanding the design, analysis, and application of different algorithms across various domains.')

with doc.create(Chapter('Algorithms')):
    doc.append('This chapter presents detailed analyses of different algorithms. It explores their mechanisms, use cases, and the theoretical concepts that underlie their effectiveness. Each algorithm is discussed with practical insights to highlight its strengths and potential limitations.')

# Loop through files in the 'python_code' directory
for filename in os.listdir('python_code/'):
    name = filename.replace(".py", "")
    formatted_name = name.replace("_", " ").title()
    folder_path = "algo_docs/"
    document_path_name = folder_path + name + "_doc.tex"
    example_path_name = folder_path + name + "_example.tex"
    print(f"Running for {filename}")
    
    # Start each iteration on a new page
    doc.append(NoEscape(r'\newpage'))
    
    #make_algo_doc(name)
    #make_algo_example(name)
    python_code_path = f"python_code/{filename}"
    
    # Insert document and example text files
    doc.append(NoEscape(rf'\input{{{document_path_name}}}'))
    doc.append(NoEscape(rf'\input{{{example_path_name}}}'))
    
    # Add image directly after the example subsection in example_path_name
    image_path = f"plots/{name}_plot.png"  # Path to your PNG file
    with doc.create(Figure(position='H')) as figure:  # Use position='H'        
        figure.add_image(image_path, width=NoEscape(r'\textwidth'))
        figure.add_caption(f'Example plot of {formatted_name}.')
    
    # Optionally, add a page break if you want the next section on a new page
    doc.append(NoEscape(r'\clearpage'))
    
    # Add sub-chapter for the code listing
    with doc.create(Subsection('Code Listing')):
        doc.append(NoEscape(rf'\lstinputlisting[language=Python]{{{python_code_path}}}'))
    
    # Generate the .tex file
    doc.generate_tex("programmatic_report")
    
    # Compile to PDF
    subprocess.run(["pdflatex", "programmatic_report.tex"])




Running for optimize_trajectory.py
examples/optimize_trajectory_example.py
Finishin making prompt
Asking gpt to make example for optimize_trajectory
Finished making example for optimize_trajectory
This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./programmatic_report.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-01-16>
(/usr/share/texlive/texmf-dist/tex/latex/base/report.cls
Document Class: report 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf/tex/latex/lm/lmodern.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty)
(/usr/share/texlive/texmf-dist/tex/latex/lastpage/lastpage.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive

In [104]:
for filename in os.listdir('python_code/'):
    name = filename.replace(".py","")
    folder_path = "algo_docs/"
    document_path_name = folder_path + name + "_doc.tex"
    print(f"Running for {filename}")
    #make_algo_doc(filename, name)

Running for optimize_trajectory.py
Running for predator_prey_system.py
Running for gravitational_assist.py
Running for calculate_hohmann_transfer.py
